In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
# Load and clean data
transactions = pd.read_csv('/content/drive/MyDrive/Data_sets/transaction_data.csv')
behavior = pd.read_csv('/content/drive/MyDrive/Data_sets/purchase_behaviour.csv')
transactions['PROD_NAME'] = transactions['PROD_NAME'].str.replace('\s+', ' ', regex=True).str.strip()
merged_data = pd.merge(transactions, behavior, on='LYLTY_CARD_NBR')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:4: DeprecationWarning: invalid escape sequence '\s'
<>:4: DeprecationWarning: invalid escape sequence '\s'
<ipython-input-2-76634b7c0e00>:4: DeprecationWarning: invalid escape sequence '\s'
  transactions['PROD_NAME'] = transactions['PROD_NAME'].str.replace('\s+', ' ', regex=True).str.strip()


In [3]:
# Assume "premium" products have a 40% margin, others 20%
# Classify premium products based on keywords in PROD_NAME
merged_data['IS_PREMIUM'] = merged_data['PROD_NAME'].str.contains('Kettle|Red Rock|Doritos', case=False)
merged_data['PROFIT'] = merged_data['TOT_SALES'] * np.where(merged_data['IS_PREMIUM'], 0.4, 0.2)

# Top 3 profitable products
top_products = merged_data.groupby('PROD_NAME')['PROFIT'].sum().nlargest(3)
print("Top 3 Profitable Products (Estimate):\n-----------------\n", top_products,'\n-----------------')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Top 3 Profitable Products (Estimate):
-----------------
 PROD_NAME
Kettle Mozzarella Basil & Pesto 175g       13782.96
Doritos Cheese Supreme 330g                13356.24
Kettle Sweet Chilli And Sour Cream 175g    13212.72
Name: PROFIT, dtype: float64 
-----------------


In [4]:
from datetime import datetime

# Calculate recency (days since last purchase)
merged_data['DATE'] = pd.to_datetime(merged_data['DATE'], origin='1899-12-30', unit='D')
snapshot_date = merged_data['DATE'].max() + pd.DateOffset(days=1)
rfm_data = merged_data.groupby('LYLTY_CARD_NBR').agg({
    'DATE': lambda x: (snapshot_date - x.max()).days,
    'TXN_ID': 'count',
    'TOT_SALES': 'sum'
}).rename(columns={
    'DATE': 'Recency',
    'TXN_ID': 'Frequency',
    'TOT_SALES': 'Monetary'
})

# Normalize RFM data
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_data)
kmeans = KMeans(n_clusters=3, random_state=42)
rfm_data['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Merge with customer demographics
rfm_data = pd.merge(rfm_data, behavior, on='LYLTY_CARD_NBR')

# Identify the most loyal cluster (low recency, high frequency/spend)
loyal_cluster = rfm_data[rfm_data['Cluster'] == rfm_data['Cluster'].mode()[0]]

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
print("\nCharacteristics of Loyal Customers (RFM Analysis):")
print('--------------------------')
print(loyal_cluster[['LIFESTAGE', 'PREMIUM_CUSTOMER']].value_counts(normalize=True).head(10))
print('--------------------------')


Characteristics of Loyal Customers (RFM Analysis):
--------------------------
LIFESTAGE               PREMIUM_CUSTOMER
YOUNG SINGLES/COUPLES   Mainstream          0.131160
RETIREES                Mainstream          0.097106
OLDER SINGLES/COUPLES   Mainstream          0.069137
                        Budget              0.068017
                        Premium             0.066685
RETIREES                Budget              0.066412
YOUNG SINGLES/COUPLES   Budget              0.059541
RETIREES                Premium             0.058179
MIDAGE SINGLES/COUPLES  Mainstream          0.048281
OLDER FAMILIES          Budget              0.047463
Name: proportion, dtype: float64
--------------------------


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# Hypothesis Generation
# Extract top products bought by loyal customers
merged_data['loyalty_cluster'] = rfm_data['Cluster']

loyal_products = merged_data[merged_data['loyalty_cluster'] == 0]['PROD_NAME'].value_counts().head(3)

# Extract demographics of loyal customers
loyal_demographics = loyal_cluster[['LIFESTAGE', 'PREMIUM_CUSTOMER']].value_counts(normalize=True).head(1)

# Generate hypothesis
hypothesis = f"""
----------------------------------------------------------------
Hypothesis:
Loyal customers (primarily {loyal_demographics.index[0][0]} and {loyal_demographics.index[0][1]} buyers)
prefer products like {loyal_products.index[0]}, {loyal_products.index[1]}, and {loyal_products.index[2]} because:
1. Quality & Brand Trust: These products are perceived as premium (e.g., "Kettle", "Doritos") and align with the preferences of {loyal_demographics.index[0][1]} buyers.
2. Usage Occasions: Older families likely purchase these snacks for shared moments (e.g., family gatherings).
3. Consistency: Loyal customers repeatedly buy these products due to reliable taste and availability.
----------------------------------------------------------------
"""

print(hypothesis)


----------------------------------------------------------------
Hypothesis:
Loyal customers (primarily YOUNG SINGLES/COUPLES and Mainstream buyers)
prefer products like Infzns Crn Crnchers Tangy Gcamole 110g, Kettle Tortilla ChpsHny&Jlpno Chili 150g, and Smiths Crnkle Chip Orgnl Big Bag 380g because:
1. Quality & Brand Trust: These products are perceived as premium (e.g., "Kettle", "Doritos") and align with the preferences of Mainstream buyers.
2. Usage Occasions: Older families likely purchase these snacks for shared moments (e.g., family gatherings).
3. Consistency: Loyal customers repeatedly buy these products due to reliable taste and availability.
----------------------------------------------------------------



/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
